In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json
import pandas as pd
from ib_async import IB, util

from ngv_reports_ibkr.download_trades import fetch_report
import os
import json

ib_json_str = os.environ["IB_JSON"]
ib_json = json.loads(ib_json_str)

In [ ]:
util.startLoop()

host = "127.0.0.1"
port = 7496
clientId = 101

TWS_CONNECTION = IB().connect(host, port, clientId, timeout=30)

In [ ]:
from ngv_reports_ibkr.download_trades import fetch_report

ib_json = json.loads(ib_json_str)

flex_token = ib_json["accounts"][0]["flex_token"]
query_id = ib_json["accounts"][0]["weekly"]

report = fetch_report(flex_token, query_id, cache_report_on_disk=False)
account_ids = report.account_ids()
ACCOUNT_ID = account_ids[2]
flex_trades_df = report.trades_by_account_id(ACCOUNT_ID)

In [ ]:
flex_trades_df.to_csv("../data/snapshots/flex_trades_df.csv")

In [ ]:
from typing import List
from abc import ABC


class RealTimeSource(ABC):
    pass


class IbkrTws(RealTimeSource):
    def __init__(self, ib=None):
        self.ib = ib
        # self.ib.reqMarketDataType(MarketDataType.LIVE.value)

    def get_accounts(self):
        return self.ib.managedAccounts()

    def get_positions_for_account(self, account_id: str):
        return self.ib.positions(account=account_id)

    def get_quotes(self, con_ids: List[str]):
        pass

    def get_trades(self):
        return self.ib.trades()

    def get_executions(self):
        return self.ib.executions()


ibkr_realtime = IbkrTws(ib=TWS_CONNECTION)

trades_ibasync_objects = ibkr_realtime.get_trades()
executions_ibasync_objects = ibkr_realtime.get_executions()
positions_ibasync_objects = ibkr_realtime.get_positions_for_account(ACCOUNT_ID)

In [ ]:
from ib_async import util

real_time_trades_df = util.df(trades_ibasync_objects)

from ngv_reports_ibkr.expand_contract_columns import expand_all_trade_columns, expand_fills_and_logs
from ngv_reports_ibkr.transforms import Transforms

expanded_df = expand_all_trade_columns(real_time_trades_df)
expanded_df = expand_fills_and_logs(expanded_df, fills_col="fills", log_col="log")

# Filter to only actual executions (removes log-only rows)
realtime_trades_df = Transforms.filter_to_executions(expanded_df)

# from pprint import pprint
# # For a single object
# pprint(vars(trades_ibasync_objects[10]))

In [ ]:
pd.set_option("display.max_rows", None)
realtime_trades_df.to_csv("../data/snapshots/realtime_trades_df.csv")